Though this is not a notebook competition, many kagglers have contributed so many wonderful notebooks. I am sure post the results announcement we will see even more emerging. This competition has larger social ramifications and I was particularly moved by one of the comment in a notebook which was by someone who wanted to contribute badly but was new to the neural nets and was struggling. This is a very small fun notebook which can serve as a headstart. I have also shared the videos, blogs and other resources which helped me a lot

Note: There is nothing new in this notebook. It just takes a couple of popular models published so far and explains it in a simple manner to someone who is totally new to neural nets/Keras/tensforflow etc. This is not aimed at only this competition but hopefully serves as a short tutorial for anybody who has an understanding of the traditional models and wants to move on to neural nets and more complicated DNN models.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
np.set_printoptions(threshold=np.inf)

inpcols=['sequence', 'structure', 'predicted_loop_type']
outcols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']

train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)

We will evolve the code as we go along as this may be the best way to understand the same. Let us start with the minimum code needed.

```model = tf.keras.Model(inputs=inp, outputs=out)```

Surely this is as straightforward as can be. We have a model and an input and an output. The input is our set of features and the output are the labels. The model - well it is a model. We havent yet defined what type of model it is but before we do that we need to add another line of code to make this more complete. If you google the good old 3 Blue 1 Brown video describing Neural networks in the simplest terms possible (https://www.youtube.com/watch?v=aircAruvnKk) , you will know that the model while training, tries to assign weights and biases to each of the neuron such that the loss is minimal. We have to define what this loss or error function is and also have to define how we want the weights and balances to be adjusted. (Well the model most certainly could do that itself but perhapes it wants us to feel we have an illusion of control..just joking). So let us go ahead and add this line to make it more complete:

```model.compile(tf.optimizers.Adam(), loss=MSE)```

We define the loss function and the weight adjustment algorithm. The weight adjustment algorithm can be Adam or Eve or RMSProp or any other optimizer, it may not make too much of a difference in this case but you can experiment and find out. The error loss function needs to be fine tuned to the problem at hand. For now we will leave it to be the mean square error but we can fine tune it later

Ok. That is pretty much it. Now let us focus on the inp and the o/ps. 

```inp = Input(shape=(107, 3))```

```out = Dense(3, activation='linear')(XXXXXXXXX)```

Input() is used to instantiate a Keras tensor. There are 107 bases and 3 i/p columns - sequence, structure and predictedloop. The number of rows is optional and need not be specified. Tensors are the units in which transactions and bargains happen in the Tensorflow Keras marketplace. A input tensor is shot thru' multiple layer of algorithms in the hope that it may strike the target accurately. It is a 'symbolic' object right now because we have only fed it certain attributes but not the real values.

Interestingly enough 'out' is not so easy to define as we did inp. In the above case 'out' is actually the ouput of the laaaaaaaaast layer of the neural net. We havent defined the intermediate layers yet. But let us define the last layer and then we will evolve the code. If XXXXXXXXXXX is the o/p of the (last-1) layer then the final o/p of the model takes XXXXXXXXX as the input into the final layer which is a Dense one (every i/p neuron is connected to every o/p neuron). There are 3 o/p neurons.. one for each of the 3 labels to be predicted. The problem is one of regression so the 'activation' to be used here can be 'linear'. 

So basically the above statement says - Take XXXXXXXX, pass it thru' a dense layer of  neurons and give me 3 o/ps and make sure these o/ps are generated by using linear activation. In case you are wondering why there would be 'non-Dense' layers in neural networks - well it helps to avoid overfitting. It is observed that randomly turning off the activation of some neurons in a layer during training helps in acheiving better scores..see (http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf). This neat technique literally 'forces' the model to learn new representations and in general avoids overfitting. There are algorithms to calculate which 'random' neurons to turn off but that is a separate story. Anyway for the final layer let us not have any hanky-panky and connect all i/p neurons of the (last-1) layer to all the 3 o/p layer neurons.

So we defined the i/p (well not completely...we need to conver the AUCGs into embeddings but we will come to that later), we defined the o/ps and we defined the model...or at least the last layer of the model. It would be a bit of a let-down if we talk of neural nets having 1 layer..the typical expectation is that we will go deeeeep, so let us define a few layers for this mode. We will select LSTM (why LSTM? it retains some memory of the sequence but feel free to use any other model..most may give similar results) and let us have 3 layers (why 3 - why not 2 or 4? again remember the illusion of being in control..hey I am joking). So:

```hidden = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(inp)```

```hidden = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(hidden)```

The intermediate layers are hidden away from the spotlight. The first hidden layer takes the i/p:inp and gives the o/p which is a variable called 'hidden'. This is fed as i/p to the next hidden layer and it gives the o/p 'hidden'. This is nothing but XXXXXXXX what we discussed earlier and is fed to the 3rd and last layer:

```out = Dense(, activation='linear')(hidden)```

As can be seen the hidden layers are straightforward to understand. By now you should not be asking questions like why dropout should be 0.3 and not 0.25 or 0.35 :) 

The 128 refers to the number of hidden dimensions which is nothing but the neurons you would like in the hidden layer. Why 128 and why not 256 or 64? Some nice people have come out with some nice rules like - The number of hidden neurons should be between the size of the input layer and the size of the output layer; The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer; The number of hidden neurons should be less than twice the size of the input layer etc etc. People always love to come out with rules. But I also loved the plain-speaking approach here: http://www.faqs.org/faqs/ai-faq/neural-nets/part3/section-10.html which states that the only rule is 'there is no rule' and you should just contextualize based on the problem at hand. quote "These rules of thumb are nonsense because they ignore the number of training
cases, the amount of noise in the targets, and the complexity of the function........"unquote.
At the end of the day go by your gut feel (in turn this depends on your past experience..if you dont have that, then rely on the GMs who have published some baseline notebooks) and please look at the CV and problem statement, data etc and tune accordingly by marking all those as hyperparameters.. We will revisit this discussion in a while though..

The 'return sequences' return the hidden state output for each input time step unlike the 'return state' which returns the hidden state output and cell state for the last input time step. By default, return_sequences is False which can be set to True. Given this problem is not very data-intensive, we should be fine and this may end up helping the model

There - we are done with the model. Well almost. Remember we havent yet converted the i/p's to embeddings. 

In [ ]:
print(train[inpcols].head(1))

Let us just map each unique character to a simple integer as a first step. In the second step, we will embed this into a vector using a embedding layer. Let us do the first step first. We can even do a 1-hot encoding in case we get better results. 

Most of the unique characters are visible above. There is only the 'M missing from the predictedlooptype. Here is a list of unique characters: AUCG and .() and ESHBXIM

In [ ]:
def cust_vectorizer(inp, characters='AUCG.()ESHBXIM'):
    return [[0 if char != letter else 1 for char in characters] for letter in inp]

mapped_num= {x:i for i, x in enumerate('AUCG.()ESHBXIM')}
mapped_OHE=dict(zip(list('AUCG.()ESHBXIM'), zip(*cust_vectorizer('AUCG.()ESHBXIM'))))

print('Simple integer Mapping:\n', mapped_num,'\n\nOHE mapping:\n',mapped_OHE)

Let us apply the simple integer mapping for now and go ahead. We sort of flatten the dataframe

In [ ]:
train=np.array(train[inpcols].applymap(lambda x: [mapped_num[item] for item in x]).values.tolist())
print(train[0], '\n\nShape of the input:\n', train.shape)

So we have 2400 rows and each row contains 3 arrays - the sequence representation, the structure representation and the predictedloop FOR EACH OF THE 107 bases. This is our input. 

We have one last thing to do here. We need to explode the array and create 2400 into 107 = 256800 rows. Because we have to predict the labels for each individual base. and while we are at it, we also need to combine the values of sequence, structure and predictedloop into one array with 3 columns. 

So we need to have a new array which has 256800 rows of input. Each row should contain an array of 3 columns - the first indicating the sequence the second the structure andf the 3rd would be the predicted loop...so an array of shape 2400 into 107 into 3. 

The transpose function can be used to change the shape from 2400,3,107 to 2400,107,3. The 2nd and 3rd dimensions are swapped

In [ ]:
train=np.transpose(train,(0, 2, 1))
print(train[0], '\n\nNew shape of the input:\n', train.shape)

So we have a 3 col vector for each of the 107 bases for each of the 2400 rows. This is our input in the right format. All we need to do here is to create an embedding for each data. Let us create a 100 dimension embedding because 100 is a nice rounded number. This terrific talk by Jay Allamar http://jalammar.github.io/skipgram-recommender-talk/ gives the best introduction to word embeddings ever possible.

We create an Embedding layer - the input dimension was 14 (for the 14 unique characters) and the o/p dimension=100

```embed = Embedding(input_dim=14, output_dim=100)(inputs)```

This will be the FIRST layer of the neural net.

In [ ]:
inp = tf.keras.layers.Input(shape=(107,3))
embedding = tf.keras.layers.Embedding(14,100)(inp)
print(embedding)

So for each of the 107 bases, we get one 100 col vector for the sequence, structure and predictedloop respectively (for all rows). One last thing to do. Let us combine the sequence, structure and predictedloop into one vector. So our final shape of the input tensor would be:

In [ ]:
inpfinal = tf.keras.backend.reshape(embedding,(-1,107,300))
print(inpfinal)

We are pretty much done with our model and have evolved the code completely. This is the final code. Of course it needs to be put in a method and parameters passed to and fro etc..

In [ ]:
##Define root mean square error
def rmse(y_actual, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_actual, y_pred)
    return keras.backend.sqrt(mse)

##Define input
inp = tf.keras.layers.Input(shape=(107,3))
embedding = tf.keras.layers.Embedding(14,100)(inp)
inpfinal = tf.keras.backend.reshape(embedding,(-1,107,300))

##Define model
hidden = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,dropout=.3,return_sequences=True))(inpfinal)
hidden = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,dropout=.3,return_sequences=True))(hidden)
hidden = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,dropout=.3,return_sequences=True))(hidden)

##Output (ignore the markers...only the first 68 are scored)
out = tf.keras.layers.Dense(3, activation='linear')(hidden[:, :68])

model = tf.keras.Model(inputs=inp, outputs=out)
model.compile(tf.optimizers.Adam(), loss=rmse)
print(model.summary())

Of course we havent actually passed any data as input to the model above. We have defined just how the model should look like and what are the inputs and outputs. Defining a model in such a abstract way enables code underneath to create computational graphs which when combined with the actual i/p data are finally run in a different language altogether (powerful C++ code) using multiple CPUs and GPUs.

The good thing is that all these layers and even models become callable objects. They have i/p, o/p and become reusable. You can treat any model as if it were a layer by invoking it on an Input or on the output of another layer. By calling a model you aren't just reusing the architecture of the model, you're also reusing its weights.

This is pretty much it. Our first attempt at building a NN model in Keras to solve the open vaccine challenge. What is left is to just pass the training data and fit the model and then do the predictions the traditional way in another 2-3 lines of code. We then get a baseline score. Then the trick is to start playing with the features and hyperparameters and see what improves the score. These are also as straightforward. Most are 1-line changes:

- Play around with the loss function. I think it is well covered in the discussions
- Add new features. Lots of new features suggested by many Kagglers, some harmless, some useful and some outright dangerous :)
- Play with the embeddings. Try OHE to see if it improves. Change the number of dimensions from 100 to 200 or 50 and if someone asks you why you are doing so pretend to be very busy with teh experiments :)
- Add a dropout layer after the embeddings to regularize
- Play around with more layers or less layers 
- Play around with more neurons and less neurons
- Try other algorithms other than LSTM (Keras supports all..)
- Augment the data. Feed the reverse of the sequence and see what happens. Generate new alternate secondary structures and loops and see if it helps
- Play around with the noise filters to remove noisy data or retain them if you feel your neural net is capable of ignoring them
- See if you can reuse any existing embeddings instead of creating from scratch
- If you are enterprising enough, try wavenets, 1D convolutions, Graph convolutions (GCN), custom transformers (well these would definitely be more than 1 line of change)

Coming back to the question of how many layers and how many neurons to use, you can try the stretch pant approach. Go for a larger size than what you need and then regularize (say by using dropout) till the pants shrink and fit you.

I will try to do some of the above and share if time permits and if this notebook turns out to be of use to anyone.

If this is your first neural net program you may be awestruck by its immense potential. With a few lines of code we acheieve something really dramatic. While there are are many great introductions to neural nets, nothing can beat this wonderful video by late Prof Patrick Winston at MIT: https://www.youtube.com/watch?v=uXt8qF2Zzfo

All the best on your Deep Neural net journey!